In [1]:
import numpy as np
import itertools
import pandas as pd 
from itertools import product
import random
from six.moves import cPickle as pickle

import gc


# data_root = "C:\\Users\\Tigran PC\\Desktop\\MicrobiomicsData\\data\\" # notebook
# data_root = "/Users/tigran/Desktop/sbv/data/" # imac
data_root = "C:\\Users\\Administrator\\Repos\\Microbiomics\\data\\" # server

In [22]:
def make_sub_seq_bank(initial_string, sub_seq_len):
    return [''.join(tup) for tup in  list(set(product(set(initial_string), repeat = sub_seq_len)))]

    
def convert_read_to_numb(dataframe, sub_str_len):
    ss_bank =  make_sub_seq_bank("ATCG", sub_str_len)
    data_list = []
    for seq in dataframe.values:
        sub_seq_count = []
        for ss in ss_bank:
            sub_seq_count.append(seq[0].count(ss))
        sub_seq_count.append(seq[1])      
        data_list.append(sub_seq_count)
    return pd.DataFrame(data_list, columns=ss_bank.append("lable")) 

def convert_read_to_numb_map(dataframe, sub_str_len):
    ss_bank =  make_sub_seq_bank("ATCG", sub_str_len)
    data_list = []
    for seq in dataframe.values:
        sub_seq_count = list(map(seq[0].count, ss_bank))
        sub_seq_count.append(seq[1])      
        data_list.append(sub_seq_count)
    return pd.DataFrame(data_list, columns=ss_bank.append("lable"))  


In [3]:
# df1 = pd.read_pickle(data_root + "DNA_data1.pickle")

In [23]:
# df2 = pd.read_pickle(data_root + "DNA_data2.pickle")

In [24]:
# print(df1.shape)
# print(df2.shape)
# df = pd.concat([df1, df2]) 
# del df1, df2
# df.head()

In [25]:
print(df.shape)
df.head()

(78731623, 2)


,0,lable
0,TTACGACACCTGGTCGACGGTGTACAACCAGCTCGAGGGCACTTGG...,1768
1,GAACGGCCGCAGCTCGTCGATCACCCCGGTCAGCGCCCGCGTCTCC...,1751294
2,AGCCTATATTAAATTGCTACCGCCTGAAAAACGAGGAGCGGAGAAC...,189834
3,TGGTATTTCTCTGTATCAAAATTGGCGTTTTGATAATAATACGGGA...,1954172
4,GTAAAGGTCAGTACGGCCACGTGGTCTTCACGCTCGAACCATTGCC...,741091


In [46]:
step = 1000000

In [47]:
df_s = pd.DataFrame(df.values[:step, :])

In [48]:
df_s.shape

(1000000, 2)

### Convert to numbers

In [49]:
gc.collect()
df_s.shape

(1000000, 2)

In [50]:
%time df_s1 =  convert_read_to_numb(df_s, 4) 

Wall time: 6min 51s


In [43]:
%time df_s2 =  convert_read_to_numb_map(df_s, 4) 

Wall time: 33.7 s


In [44]:
df_s1.head(3)

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
0,0,0,0,1,1,0,2,0,1,0,...,0,0,1,1,0,1,0,1,1,1768
1,1,2,0,0,2,0,2,0,0,0,...,1,1,4,0,0,0,0,3,0,1751294
2,1,1,1,1,0,0,0,3,0,0,...,3,0,0,1,0,0,0,2,0,189834


In [ ]:
columns = list(df_s.columns.values)
print(type(columns))
df_s = df_s.sample(frac=1).reset_index(drop=True)
print(df_s.shape)
df_s = df_s.drop_duplicates(subset=columns[:-1])
print(df_s.shape)

In [ ]:
pickle_file = data_root + "F_Bank_1.pickle"

try:   
    f = open(pickle_file, 'wb')
    pickle.dump(df_s, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise